In [1]:
! pip install -q datasets
! pip install -q transformers
! pip install -q accelerate
! pip install -q bitsandbytes
! pip install -q peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers.trainer_utils import set_seed

set_seed(2)

In [3]:
from pprint import pprint
from datasets import load_dataset

train_dataset = load_dataset("medmcqa", split="train")
valid_dataset = load_dataset("medmcqa", split="validation")
pprint(train_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

{'choice_type': 'single',
 'cop': 2,
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic '
        'hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or '
        'functional disorders cause hydronephrosis which by definition is used '
        'to describe dilatation of renal pelvis and calculus associated with '
        'progressive atrophy of the kidney due to obstruction to the outflow '
        'of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia '
             'can lead to the following change in kidney parenchyma',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract'}


In [4]:
train_dataset = train_dataset.rename_column("cop", "label")
valid_dataset = valid_dataset.rename_column("cop", "label")

In [5]:
pprint(train_dataset.features)

{'choice_type': Value(dtype='string', id=None),
 'exp': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'label': ClassLabel(names=['a', 'b', 'c', 'd'], id=None),
 'opa': Value(dtype='string', id=None),
 'opb': Value(dtype='string', id=None),
 'opc': Value(dtype='string', id=None),
 'opd': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'subject_name': Value(dtype='string', id=None),
 'topic_name': Value(dtype='string', id=None)}


In [6]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
from collections import Counter
import matplotlib.pyplot as plt
from datasets import Dataset
from tqdm import tqdm

plt.rcParams["font.size"] = 18

In [8]:
# def visualize_question_length(dataset: Dataset):
#     length_counter = Counter()
#     for data in tqdm(dataset):
#         length = len(tokenizer.tokenize(data["question"]))
#         length_counter[length] += 1

#     plt.bar(length_counter.keys(), length_counter.values(), width=1.0)
#     plt.xlabel("Number of tokens")
#     plt.ylabel("Number of samples")
#     plt.show()

In [9]:
# visualize_question_length(train_dataset)

In [10]:
# visualize_question_length(valid_dataset)

In [11]:
# from typing import List

# def visualize_choice_length(dataset: Dataset, choice_cols: List[str]):
#     length_counter = Counter()
#     for data in tqdm(dataset):
#         for col in choice_cols:
#             length = len(tokenizer.tokenize(data[col]))
#             length_counter[length] += 1

#     plt.bar(length_counter.keys(), length_counter.values(), width=1.0)
#     plt.xlabel("Number of tokens")
#     plt.ylabel("Number of samples")
#     plt.show()

In [12]:
# visualize_choice_length(train_dataset, ['opa', 'opb', 'opc', 'opd'])


In [13]:
# visualize_choice_length(valid_dataset, ['opa', 'opb', 'opc', 'opd'])


In [14]:
from transformers import BatchEncoding

choice_list = ['opa', 'opb', 'opc', 'opd']

def preprocess_multiple_choice(example: dict[str, str], ) -> BatchEncoding:
    num_choices = len(choice_list)

    example_choice_list = [example[c] for c in choice_list]

    repeated_question_list = [example["question"]] * num_choices
    encoded_example = tokenizer(repeated_question_list,
                                example_choice_list,
                                max_length=64)

    if "label" in example:
        encoded_example["labels"] = example["label"]
    return encoded_example

In [15]:
from transformers import AutoTokenizer

example = train_dataset[0]

encoded_example = preprocess_multiple_choice(example)
for choice in range(len(choice_list)):
    print(tokenizer.decode(encoded_example["input_ids"][choice]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[CLS] chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma [SEP] hyperplasia [SEP]
[CLS] chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma [SEP] hyperophy [SEP]
[CLS] chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma [SEP] atrophy [SEP]
[CLS] chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma [SEP] dyplasia [SEP]


In [16]:
encoded_train_dataset = train_dataset.map(preprocess_multiple_choice,
                                          remove_columns=train_dataset.column_names,)

encoded_valid_dataset = valid_dataset.map(preprocess_multiple_choice,
                                          remove_columns=valid_dataset.column_names,)

Map:   0%|          | 0/182822 [00:00<?, ? examples/s]

Map:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [17]:
print(encoded_train_dataset[0])

{'input_ids': [[101, 11888, 24471, 11031, 7941, 27208, 2349, 2000, 28378, 26113, 12070, 23760, 24759, 15396, 2064, 2599, 2000, 1996, 2206, 2689, 1999, 14234, 11968, 2368, 11714, 2863, 102, 23760, 24759, 15396, 102], [101, 11888, 24471, 11031, 7941, 27208, 2349, 2000, 28378, 26113, 12070, 23760, 24759, 15396, 2064, 2599, 2000, 1996, 2206, 2689, 1999, 14234, 11968, 2368, 11714, 2863, 102, 23760, 7361, 10536, 102], [101, 11888, 24471, 11031, 7941, 27208, 2349, 2000, 28378, 26113, 12070, 23760, 24759, 15396, 2064, 2599, 2000, 1996, 2206, 2689, 1999, 14234, 11968, 2368, 11714, 2863, 102, 2012, 18981, 10536, 102], [101, 11888, 24471, 11031, 7941, 27208, 2349, 2000, 28378, 26113, 12070, 23760, 24759, 15396, 2064, 2599, 2000, 1996, 2206, 2689, 1999, 14234, 11968, 2368, 11714, 2863, 102, 1040, 22571, 8523, 2401, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
import torch
from transformers import BatchEncoding

def collate_fn_multiple_choice(features: list[BatchEncoding],) -> dict[str, torch.Tensor]:

    label_name = "labels"

    batch_size = len(features)
    num_choices = len(features[0]["input_ids"])

    flat_features = []
    for feature in features:
        flat_features += [
            {k: v[i] for k, v in feature.items() if k != label_name}
            for i in range(num_choices)
        ]

    flat_batch = tokenizer.pad(flat_features, return_tensors="pt")

    batch = {
        k: v.view(batch_size, num_choices, -1)
        for k, v in flat_batch.items()
    }

    if label_name in features[0]:
        labels = [feature[label_name] for feature in features]
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
    return batch


In [19]:
batch_size = 4
encoded_examples = [preprocess_multiple_choice(train_dataset[i]) for i in range(batch_size)]

batch = collate_fn_multiple_choice(encoded_examples)
pprint({name: tensor.size() for name, tensor in batch.items()})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'attention_mask': torch.Size([4, 4, 64]),
 'input_ids': torch.Size([4, 4, 64]),
 'labels': torch.Size([4]),
 'token_type_ids': torch.Size([4, 4, 64])}


In [20]:
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained(model_name,
                                                   num_labels=train_dataset.features["label"].num_classes,)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model(**batch)

MultipleChoiceModelOutput(loss=tensor(1.3531, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0330,  0.0072,  0.0439,  0.0133],
        [ 0.0201,  0.0131,  0.0061,  0.0243],
        [ 0.0077,  0.0063,  0.0478,  0.1878],
        [-0.0143, -0.0013, -0.0110,  0.0306]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [22]:
from transformers import TrainingArguments

batch_size = 64

training_args = TrainingArguments(output_dir="bert_mcq",
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  learning_rate=5e-5,
                                  lr_scheduler_type="linear",
                                  warmup_ratio=0.1,
                                  num_train_epochs=1,
                                  save_strategy="steps",
                                  logging_strategy="steps",
                                  load_best_model_at_end=True,
                                  metric_for_best_model="accuracy",
                                  save_steps=1000,
                                  eval_steps=1000,
                                  evaluation_strategy="steps",
                                  fp16=True,
                                  push_to_hub=True,
                                  # bf16=True,
                                  )

In [23]:
import numpy as np

def compute_accuracy(eval_pred: tuple[np.ndarray, np.ndarray]) -> dict[str, float]:
    predictions, labels  = eval_pred
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == labels).astype(np.float32).mean().item()}

In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
    data_collator=collate_fn_multiple_choice,
    args=training_args,
    compute_metrics=compute_accuracy,)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [25]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
1000,1.359600,1.351469,0.319149
2000,1.325800,1.349126,0.335166


TrainOutput(global_step=2857, training_loss=1.340279605533011, metrics={'train_runtime': 1933.5121, 'train_samples_per_second': 94.554, 'train_steps_per_second': 1.478, 'total_flos': 2.3943680388350976e+16, 'train_loss': 1.340279605533011, 'epoch': 1.0})

In [26]:
eval_metrics = trainer.evaluate(encoded_valid_dataset)
pprint(eval_metrics)

{'epoch': 1.0,
 'eval_accuracy': 0.33516615629196167,
 'eval_loss': 1.3491263389587402,
 'eval_runtime': 15.2708,
 'eval_samples_per_second': 273.921,
 'eval_steps_per_second': 4.322}


In [29]:
tokenizer.push_to_hub(repo_id="MohamedAhmedAE/bert_mcq")
trainer.push_to_hub()

README.md:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MohamedAhmedAE/bert_mcq/commit/5cf8c1906ae37db3ae663907cd6992035f38ffe9', commit_message='Upload tokenizer', commit_description='', oid='5cf8c1906ae37db3ae663907cd6992035f38ffe9', pr_url=None, pr_revision=None, pr_num=None)